Our goal is to merge three datasets , one extracted from the documents and the second is scrapped from wikipedia using the species extracted from the documents are keywords and the third one is using the species scrapped from the plant list as keywords

In [1]:
import pandas as pd

In [2]:
data1 = pd.read_csv('scrapped_wikidata.csv')
data2=pd.read_csv('species_and_titles_dataframe.csv')

In [3]:
data1

,espèce,description
0,Schefflera abyssinica,Le genre Schefflera constitue un groupe d’arbr...
1,Schefflera Mannii,Schefflera mannii (Hook.f.) Harms est une espè...
2,Tessmannia africana,Tessmannia est un genre de plantes dicotylédon...
3,Alternanthera sessilis,Alternanthera est un genre qui regroupe plus d...
4,Pimpinella Ledermannii,Pimpinella ledermannii H. Wolff est une espèce...
...,...,...
141,Monopetalanthus Hedinii,Aphanocalyx hedinii est une espèce d'arbres de...
142,Monopetalanthus microphyllus,Monopetalanthus heitzii est une espèce de plan...
143,Deinbollia dasybotrys,Deinbollia est un genre de plantes de la famil...
144,Hydrocotyle sibthorpioides,Les hydrocotyles (Hydrocotyle) forment un genr...


In [4]:
data2

,espèce,description
0,Schefflera abyssinica,1. Schefflera abyssinica
1,Schefflera Mannii,3. Schefflera Mannii (Hooker fil.) Harms
2,Hydrocotyle sibthorpioides,1. Hydrocotyle sibthorpioides Lamarck
3,Hydrocotyle hirta,2. Hydrocotyle hirta R. Brown ex A. Richard
4,Hydrocotyle bonariensis,3. Hydrocotyle bonariensis Lamarck
...,...,...
200,Monopetalanthus Pellegrinii,5. Monopetalanthus Pellegrinii A. Chevalier
201,Swartzia fistuloides,1. Swartzia fistuloides Harms
202,Swartzia madagascariensis,2. Swartzia madagascariensis Desvaux
203,Csesalpinia Welwitschiana,1. Csesalpinia Welwitschiana (Oliver) Brenan


In [5]:
data=pd.concat([data1,data2]).reset_index(drop=True)

In [6]:
data

,espèce,description
0,Schefflera abyssinica,Le genre Schefflera constitue un groupe d’arbr...
1,Schefflera Mannii,Schefflera mannii (Hook.f.) Harms est une espè...
2,Tessmannia africana,Tessmannia est un genre de plantes dicotylédon...
3,Alternanthera sessilis,Alternanthera est un genre qui regroupe plus d...
4,Pimpinella Ledermannii,Pimpinella ledermannii H. Wolff est une espèce...
...,...,...
346,Monopetalanthus Pellegrinii,5. Monopetalanthus Pellegrinii A. Chevalier
347,Swartzia fistuloides,1. Swartzia fistuloides Harms
348,Swartzia madagascariensis,2. Swartzia madagascariensis Desvaux
349,Csesalpinia Welwitschiana,1. Csesalpinia Welwitschiana (Oliver) Brenan


In [7]:
import nltk
import random
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def TOKENIZE(Text):
    text=word_tokenize(Text)
    return(' '.join(text))
def tokenrandom(text,data,eps):
    seps=[',',';','\n',' ']
    i=-1
    if(random.uniform(0, 1)<eps):
        for i,row in data.iterrows():
            sep=random.choice(seps)
            text=row["description"]+sep+text
            data=data.drop(i)
            break
    return(TOKENIZE(text),data,i)
    

[nltk_data] Downloading package punkt to C:\Users\ATLAS PRO
[nltk_data]     ELECTRO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import re
TRAIN_DATA=[]
df=data.copy()
for idx,row in data.iterrows():
    entities=[]
    if (row['description'].lower().find(row['espèce'].lower()))!=-1:
        description,df,i=tokenrandom(row['description'].lower().replace('.','').strip(),df,0.05)
    else:
        continue
    first_indexes1=[m.start() for m in re.finditer(row['espèce'].lower(), description)]
    for fr in first_indexes1:
        entities.append([fr,fr+len(row['espèce']),'espèce'])
    try:
        first_indexes2=[m.start() for m in re.finditer(data['espèce'][i].lower(), description)]
        for fr in first_indexes2:
            entities.append([fr,fr+len(data['espèce'][i]),'espèce'])

    except Exception as e:
        print(e)
    if entities:
        TRAIN_DATA.append([description,{"entities":entities}])

well the first problem that we faced with the other models we have created is that the model detects just one entitie a time so we need to detect multiple entities 

In [9]:
TRAIN_DATA1=TRAIN_DATA

after saving the data , we need to add more data that can helps to eal with the problem we had before

In [77]:
data3=pd.read_csv('rescued_data.csv')

In [78]:
#drop duplicates and empty rows
data3=data3.dropna()
data3=data3.drop_duplicates()
data3.reset_index(drop=True,inplace=True)
data3

,espèce,description
0,Aa achalensis,Aa est un genre de plante de la famille des Or...
1,Aa argyrolepis,Aa est un genre de plante de la famille des Or...
2,Aa aurantiaca,Echinopsis est l'un des principaux genres de l...
3,Aa colombiana,Aa est un genre de plante de la famille des Or...
4,Aa denticulata,Aa est un genre de plante de la famille des Or...
...,...,...
34786,Campanula macrostachya,Cette liste des plantes à feuilles comestibles...
34787,Campanula macrostyla,Les campanules (genre Campanula du latin campa...
34788,Campanula marchesettii,"Euphrasia, les Euphraises, est un genre de pla..."
34789,Campanula medium,La Campanule carillon (Campanula medium) est u...


In [79]:
data3=data3[data3['espèce'].str.match('^\w+\s[a-z]\w+$')].reset_index(drop=True)
data3

,espèce,description
0,Aa achalensis,Aa est un genre de plante de la famille des Or...
1,Aa argyrolepis,Aa est un genre de plante de la famille des Or...
2,Aa aurantiaca,Echinopsis est l'un des principaux genres de l...
3,Aa colombiana,Aa est un genre de plante de la famille des Or...
4,Aa denticulata,Aa est un genre de plante de la famille des Or...
...,...,...
34610,Campanula macrostachya,Cette liste des plantes à feuilles comestibles...
34611,Campanula macrostyla,Les campanules (genre Campanula du latin campa...
34612,Campanula marchesettii,"Euphrasia, les Euphraises, est un genre de pla..."
34613,Campanula medium,La Campanule carillon (Campanula medium) est u...


In [80]:
data3=data3.astype('string')
data3.dtypes

espèce         string
description    string
dtype: object

In [81]:
import nltk
import random
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def TOKENIZE(Text):
    text=word_tokenize(Text)[0:512]
    return(' '.join(text))
def tokenrandom(text,data,eps,esp_name):
    seps=[',',';','\n',' ']
    i=-1
    j=0
    if(random.uniform(0, 1)<eps):
        for i,row in data.iterrows():
            if (row['description'].lower().find(row['espèce'].lower()))!=-1 and row['espèce'].lower()!=esp_name:
                sep=random.choice(seps)
                text=row["description"]+sep+text
                data=data.drop(i)
                j=j+1
                if j>2:
                    print('*************************** alert********************')
                break  
    return(TOKENIZE(text),data,i)
    

[nltk_data] Downloading package punkt to C:\Users\ATLAS PRO
[nltk_data]     ELECTRO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [118]:
test=word_tokenize("mokaaa ll lllllll kdkdkdkdkd dkkkd ")
test[0:512]

['mokaaa', 'll', 'lllllll', 'kdkdkdkdkd', 'dkkkd']

In [53]:
data3=data3.sample(5000).reset_index(drop=True)

In [82]:
data3

,espèce,description
0,Aa achalensis,Aa est un genre de plante de la famille des Or...
1,Aa argyrolepis,Aa est un genre de plante de la famille des Or...
2,Aa aurantiaca,Echinopsis est l'un des principaux genres de l...
3,Aa colombiana,Aa est un genre de plante de la famille des Or...
4,Aa denticulata,Aa est un genre de plante de la famille des Or...
...,...,...
34610,Campanula macrostachya,Cette liste des plantes à feuilles comestibles...
34611,Campanula macrostyla,Les campanules (genre Campanula du latin campa...
34612,Campanula marchesettii,"Euphrasia, les Euphraises, est un genre de pla..."
34613,Campanula medium,La Campanule carillon (Campanula medium) est u...


In [ ]:
import re
TRAIN_DATA=[]
TRAIN_DATA2=[]
df=data3.copy()
for idx,row in data3.iterrows():
    entities=[]
    if (row['description'].lower().find(row['espèce'].lower()))!=-1:
        description,df,i=tokenrandom(row['description'].lower().replace('.','').strip(),df,0.9,row['espèce'].lower())
    else:
        continue
    first_indexes1=[m.start() for m in re.finditer(row['espèce'].lower(), description)]
    for fr in first_indexes1:
        entities.append([fr,fr+len(row['espèce']),'espèce'])
    try:
        first_indexes2=[m.start() for m in re.finditer(data3['espèce'][i].lower(), description)]
        for fr in first_indexes2:
            entities.append([fr,fr+len(data3['espèce'][i]),'espèce'])
            print(i)
        TRAIN_DATA2.append([description,{"entities":entities}])
    except Exception as e:
        i=-1
    if entities:
        TRAIN_DATA.append([description,{"entities":entities}])

In [87]:
len(TRAIN_DATA2)

2871

In [ ]:
TRAIN_DATA3=[]
for el in TRAIN_DATA2:
    if(len(el[1]['entities'])>1):
        TRAIN_DATA3.append(el)
TRAIN_DATA3

In [95]:
len(TRAIN_DATA1)

246

In [96]:
for el in TRAIN_DATA3:
    TRAIN_DATA1.append(el)

In [108]:
import pickle
with open ("train_data.pkl","wb") as pick:
    pickle.dump(TRAIN_DATA1,pick)